# Path setup

In [5]:
main_path = '../Data/videos/'

# Importations and Installations

In [6]:
import json
import os

import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd

# Imports
import os

from sklearn.linear_model    import LinearRegression

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore') # ignore some warnings


SEED = 123456789   # Define a random seed so random steps can be replicated. 


In [7]:
left_hand = ["LEFT_THUMB.x", "LEFT_INDEX.x", "LEFT_PINKY.x",  "LEFT_WRIST.x", "LEFT_THUMB.y", "LEFT_INDEX.y", "LEFT_PINKY.y",  "LEFT_WRIST.y", "LEFT_THUMB.v", "LEFT_INDEX.v", "LEFT_PINKY.v",  "LEFT_WRIST.v"]
right_hand = ["RIGHT_THUMB.x", "RIGHT_INDEX.x", "RIGHT_PINKY.x",  "RIGHT_WRIST.x", "RIGHT_THUMB.y", "RIGHT_INDEX.y", "RIGHT_PINKY.y",  "RIGHT_WRIST.y", "RIGHT_THUMB.v", "RIGHT_INDEX.v", "RIGHT_PINKY.v",  "RIGHT_WRIST.v"]
left_foot = ["LEFT_FOOT_INDEX.x", "LEFT_ANKLE.x", "LEFT_HEEL.x", "LEFT_FOOT_INDEX.y", "LEFT_ANKLE.y", "LEFT_HEEL.y", "LEFT_FOOT_INDEX.v", "LEFT_ANKLE.v", "LEFT_HEEL.v"]
right_foot = ["RIGHT_FOOT_INDEX.x", "RIGHT_ANKLE.x", "RIGHT_HEEL.x", "RIGHT_FOOT_INDEX.y", "RIGHT_ANKLE.y", "RIGHT_HEEL.y", "RIGHT_FOOT_INDEX.v", "RIGHT_ANKLE.v", "RIGHT_HEEL.v"]
extremities = {"left_hand" : left_hand, "right_hand": right_hand, "left_foot" : left_foot, "right_foot" : right_foot}

# Train model to predict position of the body landmarks from extremities

### First get pose sequence information to train the model to predict the landmarks position

In [8]:
videos = ['boulder_1_04', 'boulder_2_02', 'boulder_3_03']

df_final = pd.DataFrame()
dfs = []
for video in videos:
    data =json.load(open(main_path + video + '/' + video + '_POSE.json'))
    df = pd.json_normalize(data)
    dfs.append(df)


In [9]:
# Concatenate the dataframe into a unique one to use as a train/test set
df_final = pd.concat(dfs)
df_final

,LEFT_EYE_INNER.x,LEFT_EYE_INNER.y,LEFT_EYE_INNER.v,LEFT_EYE.x,LEFT_EYE.y,LEFT_EYE.v,LEFT_EYE_OUTER.x,LEFT_EYE_OUTER.y,LEFT_EYE_OUTER.v,RIGHT_EYE_INNER.x,...,RIGHT_HIP.v,LEFT_HIP.x,LEFT_HIP.y,LEFT_HIP.v,RIGHT_SHOULDER.x,RIGHT_SHOULDER.y,RIGHT_SHOULDER.v,LEFT_SHOULDER.x,LEFT_SHOULDER.y,LEFT_SHOULDER.v
0,0.574033,0.632629,0.999316,0.572735,0.632456,0.999181,0.571453,0.632280,0.999160,0.577669,...,0.999895,0.591523,0.607604,0.999904,0.598253,0.634587,0.999605,0.569853,0.629704,0.999765
1,0.601052,0.621455,0.999380,0.602316,0.621491,0.999259,0.603565,0.621529,0.999236,0.596943,...,0.999874,0.596516,0.605089,0.999874,0.576945,0.616103,0.999619,0.613640,0.618872,0.999763
2,0.589538,0.629733,0.999425,0.587694,0.629856,0.999308,0.585846,0.629968,0.999296,0.594478,...,0.999847,0.587995,0.595155,0.999840,0.613829,0.623784,0.999636,0.583776,0.627473,0.999769
3,0.629600,0.612378,0.999426,0.630356,0.612422,0.999278,0.630382,0.612509,0.999297,0.629757,...,0.999772,0.624270,0.663279,0.999854,0.620764,0.625918,0.999407,0.638795,0.630043,0.999747
4,0.632866,0.612649,0.999261,0.633064,0.612679,0.999093,0.633111,0.612744,0.999110,0.633617,...,0.999745,0.612128,0.665145,0.999858,0.634944,0.626695,0.999319,0.633006,0.630054,0.999752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,1.014892,0.929562,0.434201,1.015876,0.928105,0.426506,1.016831,0.926704,0.402676,1.011300,...,0.831660,0.987929,0.954359,0.778419,0.986780,0.897651,0.994460,1.025928,0.901987,0.849996
1312,1.000675,0.940998,0.431597,1.001510,0.939381,0.425284,1.002323,0.937803,0.405078,0.997467,...,0.848159,0.971342,0.950287,0.797535,0.967627,0.921801,0.994165,1.004301,0.897119,0.858585
1313,0.979813,0.941740,0.409418,0.980392,0.940917,0.402969,0.980962,0.940150,0.382002,0.978663,...,0.863060,0.962534,0.960960,0.816755,0.968952,0.929526,0.993315,0.988434,0.918927,0.861747
1314,0.992065,0.942951,0.371301,0.992881,0.942466,0.365918,0.993722,0.942027,0.346639,0.989608,...,0.876467,0.973271,0.964853,0.833865,0.973639,0.918558,0.990020,1.000075,0.927716,0.850441


In [10]:
# Setup the input and output variables for the linear regression model:
list_extremities = left_hand + right_hand + left_foot + right_foot
list_body = list(set(list(df_final.columns)) - set(list_extremities))
list_extremities = [x for x in list_extremities if ".v" not in x]
list_body = [x for x in list_body if ".v" not in x]
list_visibilities = [x for x in list(df_final.columns) if ".v" in x]

X = pd.DataFrame(df_final, columns = list_extremities)
y = pd.DataFrame(df_final, columns = list_body) 


In [11]:
# Divide data into a training set and a testing set using the train_test_split() function
#   NOTE: Please call the subsamples `X_train` ,  `X_test` , `y_train` , `y_test` so we can follow your code
indices = range(0,X.shape[0],1)
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size=0.20, random_state=SEED)

In [12]:
regr = LinearRegression()
  
regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))

0.9912075105745203


We have a model trained to predict the landmarks of the body from the landmarks of the extremities. Now, we use it to generate the skeleton of the climber from the move sequence

First, test by reading the pose sequence of a climb, removing the body landmarks, and predicting them with the model

In [13]:
main_path = '../Data/'
video = 'moonboard'

data =json.load(open(main_path + video + '/' + video + '_POSE.json'))
df = pd.json_normalize(data)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/moonboard/moonboard_POSE.json'

In [ ]:
X_pred = pd.DataFrame(df, columns = list_extremities)
y_true = pd.DataFrame(df_final, columns = list_body) 

y_pred = pd.DataFrame(regr.predict(X_pred), columns = list_body)

In [ ]:
X_pred.dropna(inplace=True)
X_pred

,LEFT_THUMB.x,LEFT_INDEX.x,LEFT_PINKY.x,LEFT_WRIST.x,LEFT_THUMB.y,LEFT_INDEX.y,LEFT_PINKY.y,LEFT_WRIST.y,RIGHT_THUMB.x,RIGHT_INDEX.x,...,LEFT_HEEL.x,LEFT_FOOT_INDEX.y,LEFT_ANKLE.y,LEFT_HEEL.y,RIGHT_FOOT_INDEX.x,RIGHT_ANKLE.x,RIGHT_HEEL.x,RIGHT_FOOT_INDEX.y,RIGHT_ANKLE.y,RIGHT_HEEL.y
0,0.248809,0.244059,0.240949,0.250727,0.699277,0.697148,0.699085,0.706191,0.471839,0.475069,...,0.455777,0.733785,0.742545,0.746385,0.531531,0.509619,0.515083,0.741120,0.747376,0.753392
1,0.246908,0.242014,0.238896,0.249044,0.698199,0.696066,0.698472,0.705304,0.471851,0.475148,...,0.455204,0.734302,0.741860,0.745580,0.524689,0.508867,0.510262,0.744789,0.748351,0.754063
2,0.245575,0.240788,0.237412,0.247877,0.697684,0.695497,0.698149,0.704999,0.471852,0.475163,...,0.459243,0.736789,0.742138,0.745975,0.525315,0.508214,0.507455,0.746440,0.748850,0.754532
3,0.245022,0.240389,0.236783,0.247347,0.697655,0.695480,0.698160,0.705024,0.471815,0.475153,...,0.455245,0.730545,0.737528,0.742064,0.521199,0.505229,0.502239,0.747408,0.749120,0.755815
4,0.244928,0.240312,0.236900,0.247327,0.697695,0.695523,0.698238,0.705048,0.472053,0.475455,...,0.451298,0.730626,0.736918,0.741132,0.522168,0.504405,0.502154,0.747250,0.749619,0.756752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,0.875260,0.872862,0.889305,0.895769,0.749762,0.755810,0.757730,0.741976,0.892994,0.887793,...,0.965325,0.877419,0.870047,0.884377,0.912134,0.969895,0.983707,0.864998,0.851281,0.862136
1066,0.935319,0.937407,0.953973,0.955581,0.748528,0.753823,0.754432,0.739959,0.897714,0.893098,...,0.973347,0.879655,0.872597,0.886967,0.905497,0.970864,0.983318,0.869160,0.856713,0.866659
1067,0.915585,0.914536,0.925897,0.931049,0.734125,0.737926,0.737891,0.723753,0.909744,0.905132,...,0.990273,0.878975,0.865555,0.880009,0.906846,0.968333,0.980962,0.866554,0.845979,0.855725
1068,0.919872,0.918675,0.926074,0.931682,0.733360,0.736934,0.737410,0.722125,0.922539,0.917312,...,1.025312,0.881633,0.859710,0.870340,0.950917,0.991596,1.013490,0.868090,0.848533,0.856770


In [ ]:
y_pred

,LEFT_EYE_OUTER.x,MOUTH_LEFT.y,RIGHT_HIP.x,RIGHT_EYE_INNER.x,RIGHT_SHOULDER.y,RIGHT_ELBOW.y,RIGHT_EAR.x,LEFT_EAR.x,RIGHT_EYE.x,LEFT_EYE.y,...,RIGHT_KNEE.y,LEFT_SHOULDER.y,LEFT_EYE_INNER.y,RIGHT_EYE_OUTER.x,MOUTH_RIGHT.x,RIGHT_HIP.y,LEFT_EYE_INNER.x,RIGHT_EYE.y,LEFT_HIP.y,LEFT_EYE.x
0,0.331846,0.683248,0.432920,0.338215,0.699122,0.703135,0.349311,0.334103,0.340006,0.675481,...,0.724730,0.712896,0.674996,0.341843,0.342422,0.753422,0.334059,0.674529,0.759240,0.332956
1,0.331019,0.679600,0.437664,0.337538,0.695179,0.700618,0.348637,0.333432,0.339387,0.671523,...,0.725203,0.709793,0.671026,0.341271,0.341773,0.754021,0.333275,0.670537,0.760059,0.332161
2,0.329047,0.677344,0.435126,0.335639,0.692537,0.699140,0.346188,0.331080,0.337450,0.668722,...,0.724501,0.706950,0.668267,0.339299,0.340154,0.753826,0.331363,0.667796,0.759016,0.330210
3,0.326872,0.676227,0.431629,0.333750,0.692313,0.699215,0.344051,0.328613,0.335591,0.667431,...,0.721662,0.705575,0.667014,0.337479,0.338196,0.754159,0.329323,0.666740,0.758450,0.328103
4,0.324445,0.676261,0.427908,0.331328,0.692172,0.698856,0.341295,0.325894,0.333129,0.667480,...,0.720870,0.704932,0.667093,0.334978,0.335936,0.752420,0.326931,0.666844,0.756477,0.325684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,0.938350,0.616001,1.014095,0.932914,0.615248,0.676385,0.931582,0.939634,0.931945,0.611375,...,0.800891,0.635273,0.610817,0.930841,0.930833,0.696300,0.936522,0.608417,0.709562,0.937502
1066,0.978459,0.627539,1.044511,0.971273,0.627956,0.673857,0.973640,0.984388,0.970451,0.622945,...,0.807471,0.648663,0.622221,0.969502,0.966855,0.714416,0.975293,0.619122,0.727129,0.976920
1067,0.977584,0.606439,1.047549,0.971198,0.608970,0.662003,0.976355,0.984768,0.970751,0.598919,...,0.792281,0.628280,0.598326,0.970228,0.968009,0.703165,0.974560,0.595648,0.714467,0.976153
1068,0.973943,0.604839,1.031308,0.970081,0.608635,0.657705,0.975655,0.979326,0.969938,0.598662,...,0.795323,0.625125,0.598218,0.969743,0.968114,0.689108,0.972167,0.596290,0.701403,0.973191


In [ ]:
# We now have a dataframe <df_pred> containing all the spatial informations of the landmarks at each frame
df_pred = pd.concat([X_pred, y_pred], axis=1)

# Append the visibilities (by default = 1) to have complete informations for the landmarks
for v in list_visibilities:
    df_pred[v] = 1

   
df_pred = df_pred.reindex(sorted(df.columns), axis=1)
df_pred

,LEFT_ANKLE.v,LEFT_ANKLE.x,LEFT_ANKLE.y,LEFT_EAR.v,LEFT_EAR.x,LEFT_EAR.y,LEFT_ELBOW.v,LEFT_ELBOW.x,LEFT_ELBOW.y,LEFT_EYE.v,...,RIGHT_PINKY.y,RIGHT_SHOULDER.v,RIGHT_SHOULDER.x,RIGHT_SHOULDER.y,RIGHT_THUMB.v,RIGHT_THUMB.x,RIGHT_THUMB.y,RIGHT_WRIST.v,RIGHT_WRIST.x,RIGHT_WRIST.y
0,1,0.450234,0.742545,1,0.334103,0.682101,1,0.274652,0.734098,1,...,0.664663,1,0.389016,0.699122,1,0.471839,0.665715,1,0.471040,0.672502
1,1,0.449988,0.741860,1,0.333432,0.678108,1,0.273954,0.731139,1,...,0.664352,1,0.388984,0.695179,1,0.471851,0.665210,1,0.471123,0.671773
2,1,0.454972,0.742138,1,0.331080,0.674973,1,0.272665,0.729863,1,...,0.664077,1,0.386599,0.692537,1,0.471852,0.664826,1,0.471165,0.671359
3,1,0.450184,0.737528,1,0.328613,0.673646,1,0.270883,0.729022,1,...,0.663917,1,0.384327,0.692313,1,0.471815,0.664608,1,0.471166,0.671160
4,1,0.446810,0.736918,1,0.325894,0.673513,1,0.269897,0.728808,1,...,0.663465,1,0.381812,0.692172,1,0.472053,0.664000,1,0.471404,0.670688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1,0.949194,0.870047,1,0.939634,0.614472,1,0.925585,0.677706,1,...,0.742554,1,0.935345,0.615248,1,0.892994,0.730355,1,0.908032,0.723362
1066,1,0.958377,0.872597,1,0.984388,0.626424,1,0.982065,0.680368,1,...,0.730329,1,0.969352,0.627956,1,0.897714,0.725459,1,0.907803,0.716232
1067,1,0.979113,0.865555,1,0.984768,0.601659,1,0.964600,0.664607,1,...,0.730406,1,0.979375,0.608970,1,0.909744,0.726109,1,0.920217,0.715499
1068,1,0.998128,0.859710,1,0.979326,0.601065,1,0.958859,0.662046,1,...,0.729902,1,0.981651,0.608635,1,0.922539,0.724895,1,0.933088,0.713077


Finally, use Mediapipe to visualize the generated skeleton

In [ ]:
# Mediapipe parameters
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
# Setup parameters to prepare the video
ordered_columns = [ 'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
                    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER',
                    'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY',
                    'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE',
                    'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX',
                    'RIGHT_FOOT_INDEX']

def draw_predicted_skeleton(video, df_pred, frame_step = 10, drawing_with_MP = True, extension = '_PRED.mp4'):

    directory = main_path + video + '/'
    img = cv2.imread(directory + video + '_SCREEN.jpg')

    frame_width = img.shape[1]
    frame_height = img.shape[0]

    fps = 60

    # Outpath
    out = cv2.VideoWriter(directory + video  + extension, cv2.VideoWriter_fourcc(*'MP4V'), fps, (frame_width, frame_height))

    # Loop over all landmarks and draw it on the video based on its informations
    
    for i in range(df_pred.shape[0]): # Loop over the frames
        if i % frame_step != 0: continue

        if i % 100 == 0: print(f"Processing frame {i}")
        
        img = cv2.imread(directory + video + '_SCREEN.jpg')

        if (drawing_with_MP):
            # Convert the predicted informations to a format that can be read as results.pose_landmarks

            landmarks_list = []

            for l in ordered_columns:
                x = df_pred[l+'.x'][i] 
                y = df_pred[l+'.y'][i] 
                z = 0
                v = df_pred[l+'.v'][i] 

                landmarks_list.append({'x' : x, 'y' : y, 'z' : z, 'visibility' : v})
            
            pose_landmarks = landmark_pb2.NormalizedLandmarkList(
                landmark = landmarks_list
            )


            # Draw the landmarks on the video
            mp_drawing.draw_landmarks(
                img,
                pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec = mp_drawing.DrawingSpec(color=(245,117,66), thickness=3, circle_radius=2),
                connection_drawing_spec = mp_drawing.DrawingSpec(color=(245,66,230), thickness=3, circle_radius=2))
                # landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        else :
            for j in range(0, df_pred.shape[1], 3):
                x = df_pred.iloc[i,j+1] 
                y = df_pred.iloc[i,j+2]
                img = cv2.circle(img, (int(x*frame_width), int(y*frame_height)), radius=10, color=(255,0,0), thickness=-1)
        
        out.write(img)

    out.release()
       


In [ ]:
# draw_predicted_skeleton(video, df_pred=df_pred, frame_step = 5, drawing_with_MP=True)

# Now, generate sequence of landmarks from the move sequence

Instead of generating the skeleton from the landmarks, first generate the landmarks from the move sequence, and then generate the skeleton from the landmarks

### First, we do it by using frames as references [OBSOLETE]

In [ ]:
# Recover the move sequence from the csv file generated by the other Notebook
# directory = main_path + video + '/'
# df_sequence = pd.read_csv(main_path+ '/' + 'moonboard' + '/' + 'move_sequence.csv')
# df_sequence = df_sequence.iloc[:,1:]
# df_sequence

In [ ]:
left_hand = ["LEFT_THUMB.x", "LEFT_INDEX.x", "LEFT_PINKY.x",  "LEFT_WRIST.x", "LEFT_THUMB.y", "LEFT_INDEX.y", "LEFT_PINKY.y",  "LEFT_WRIST.y", "LEFT_THUMB.v", "LEFT_INDEX.v", "LEFT_PINKY.v",  "LEFT_WRIST.v"]
right_hand = ["RIGHT_THUMB.x", "RIGHT_INDEX.x", "RIGHT_PINKY.x",  "RIGHT_WRIST.x", "RIGHT_THUMB.y", "RIGHT_INDEX.y", "RIGHT_PINKY.y",  "RIGHT_WRIST.y", "RIGHT_THUMB.v", "RIGHT_INDEX.v", "RIGHT_PINKY.v",  "RIGHT_WRIST.v"]
left_foot = ["LEFT_FOOT_INDEX.x", "LEFT_ANKLE.x", "LEFT_HEEL.x", "LEFT_FOOT_INDEX.y", "LEFT_ANKLE.y", "LEFT_HEEL.y", "LEFT_FOOT_INDEX.v", "LEFT_ANKLE.v", "LEFT_HEEL.v"]
right_foot = ["RIGHT_FOOT_INDEX.x", "RIGHT_ANKLE.x", "RIGHT_HEEL.x", "RIGHT_FOOT_INDEX.y", "RIGHT_ANKLE.y", "RIGHT_HEEL.y", "RIGHT_FOOT_INDEX.v", "RIGHT_ANKLE.v", "RIGHT_HEEL.v"]
extremities = {"left_hand" : left_hand, "right_hand": right_hand, "left_foot" : left_foot, "right_foot" : right_foot}

def generate_skeleton_from_move_seq(df_sequence, nb_frames):
    """ Generates the skeleton by interpolating between frames. This is not ideal
    since we want to generate it from just the holds only"""
    
    initial_nb_frames = df_sequence.iloc[-1]["frame"]
    scaling_factor = nb_frames/initial_nb_frames

    X_gen = pd.DataFrame(columns=list_extremities)
    last_frames = {"left_hand" : 0, "right_hand": 0, "left_foot" : 0, "right_foot" : 0}

    current_frame = 0
    # Initialize first position (which should be given by the first 4 frames for each extermity)
    X_gen.loc[0] = 0
    for i in range(4):
        key = df_sequence.iloc[i]["limb"]
        current_frame = df_sequence.iloc[i]["frame"]
        last_frames[key] = current_frame

        for extremity in extremities[key]:
            if(".x" in extremity):
                X_gen.iloc[0][extremity] = df_sequence.iloc[i]["x"]
            if(".y" in extremity):
                X_gen.iloc[0][extremity] = df_sequence.iloc[i]["y"]

    # Initialize all the frames from 0 to the current_frame in the df to
    # have the same sequence
    for i in range(current_frame+1):
        X_gen.loc[i] = X_gen.loc[0]

    # In case the first 4 frames are not the ones we want, initialize the missing values
    # with the mean of the coordinates 
    X_gen.fillna(X_gen.mean())

    # Now, go through the data frame and update lerp the coordinates of the limbs as they move
    
    for i in range(4, df_sequence.shape[0]):
        df_new = df_sequence.loc[i]
        # Get the current limb
        key = df_new["limb"]

        # Recover the last frame we saw it, and upadte it in the list
        old_frame = last_frames[key]
        new_frame = df_new["frame"]


        
        last_frames[key] = new_frame

        df_old = df_sequence.loc[df_sequence["frame"] == old_frame]
        df_old = df_old.loc[df_old["limb"] == key]
        
        # if df_old.empty:
        #     df_old = df_sequence.iloc[0]
        # else:
        #     df_old = df_old.iloc[0]


        # Compute the difference between the 2 frames to get an idea of the speed of motion      

        diff = new_frame - old_frame
        

        # Adjust the difference so that we have the required number of frames in the end
        new_diff = int(diff * scaling_factor)

        # # Interpolate the new coordinates over <new_diff> frames for all the extremities
        # # corresponding to the current limb
        for j in range(1,new_diff+1):
            current_frame += 1
            X_gen.loc[current_frame] = X_gen.loc[current_frame-1]


            coeff = j/new_diff
            for extremity in extremities[key]:
                if(".x" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["x"] - df_old["x"])*coeff + df_old["x"]
                if(".y" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["y"] - df_old["y"])*coeff + df_old["y"]



    return X_gen



In [ ]:
# nb_frames = 500
# X_gen = generate_skeleton_from_move_seq(df_sequence, nb_frames)



# y_gen = pd.DataFrame(regr.predict(X_gen), columns = list_body)
# df_gen = pd.concat([X_gen, y_gen], axis=1)
# df_gen.to_csv(main_path + video + '/'+ video +'_generated_skeleton_frames.csv')

# for v in list_visibilities:
#     df_gen[v] = 1

# df_gen = df_gen.reindex(sorted(df.columns), axis=1)
# df_gen

In [ ]:
# draw_predicted_skeleton(video, df_pred=df_gen, frame_step = 5, drawing_with_MP=True, extension = '_GEN_frames.mp4')

### Then, we do the same but without using the frame informations

In [ ]:
# df_sequence = df_sequence.iloc[:,1:]
video = 'moonboard'

directory = main_path + video + '/'
df_sequence = pd.read_csv(directory + '/' + video + '_MOVE_SEQ.csv')
df_sequence = df_sequence.iloc[:,1:]
df_sequence

,x,y,limb
0,0.686667,0.740816,left_hand
1,0.840000,0.738776,right_hand
2,0.678333,0.881633,left_foot
3,0.835000,0.932653,right_foot
4,0.536667,0.583673,left_hand
5,0.681667,0.876531,right_foot
6,0.461667,0.733673,left_foot
7,0.683333,0.392857,right_hand
8,0.836667,0.632653,right_foot
9,0.530000,0.244898,left_hand


In [ ]:
def generate_skeleton_from_move_seq2(df_sequence, nb_frames):
    """Generates the skeleton from the move sequence without using frame
    information. It therefore does a spatial interpolation"""

    # Compute scaling factor such that each move gets same number of frames (remove the first 4 because they're used to set the starting position)
    nb_frames_per_move = int(nb_frames/(df_sequence.shape[0]-4))


    X_gen = pd.DataFrame(columns=list_extremities, dtype=float)
    last_positions = {"left_hand" : [], "right_hand": [], "left_foot" : [], "right_foot" : []}


    current_frame = 0
    # Initialize first position (which should be given by the first 4 frames for each extermity)
    X_gen.loc[0] = 0.
    for i in range(4):
        key = df_sequence.iloc[i]["limb"]
        current_position = [df_sequence.iloc[i]["x"], df_sequence.iloc[i]["y"]]
        last_positions[key] = current_position

        for extremity in extremities[key]:
            if(".x" in extremity):
                X_gen.iloc[0][extremity] = float(df_sequence.iloc[i]["x"])
            if(".y" in extremity):
                X_gen.iloc[0][extremity] = df_sequence.iloc[i]["y"]


    # Initialize all the frames from 0 to the current_frame in the df to
    # have the same sequence
    for i in range(current_frame+1):
        X_gen.loc[i] = X_gen.loc[0]

    # In case the first 4 frames are not the ones we want, initialize the missing values
    # with the mean of the coordinates 
    X_gen.fillna(X_gen.mean())

    # Now, go through the data frame and update lerp the coordinates of the limbs as they move
    
    for i in range(4, df_sequence.shape[0]):
        df_new = df_sequence.loc[i]
        # Get the current limb
        key = df_new["limb"]

        # Recover the last frame we saw it, and upadte it in the list
        old_position= last_positions[key]
        new_position = [df_new["x"], df_new["y"]]

        last_positions[key] = new_position

        df_old = df_sequence.loc[(df_sequence["limb"] == key) & (df_sequence["x"] == old_position[0]) & (df_sequence["y"] == old_position[1])]
        # print(df_old)
        df_old = df_old.iloc[0]

        # if df_old.empty:
        #     df_old = df_sequence.iloc[0]
        # else:
        #     df_old = df_old.iloc[0]


        # Compute the distance between the 2 positions to get an idea of the speed of motion      

        dist = np.linalg.norm(np.array(new_position) - np.array(old_position))
        nb_frames_for_this_move = int(nb_frames_per_move * (dist/0.2))
        print(nb_frames_for_this_move)

        # # Adjust the difference so that we have the required number of frames in the end

        # # Interpolate the new coordinates over <new_diff> frames for all the extremities
        # # corresponding to the current limb
        for j in range(1,nb_frames_for_this_move+1):
            current_frame += 1
            X_gen.loc[current_frame] = X_gen.loc[current_frame-1]


            coeff = j/nb_frames_for_this_move
            for extremity in extremities[key]:
                if(".x" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["x"] - df_old["x"])*coeff + df_old["x"]
                if(".y" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["y"] - df_old["y"])*coeff + df_old["y"]


    return X_gen



In [ ]:
nb_frames = 1500
X_gen = generate_skeleton_from_move_seq2(df_sequence, nb_frames)


y_gen = pd.DataFrame(regr.predict(X_gen), columns = list_body)
df_gen = pd.concat([X_gen, y_gen], axis=1)
df_gen.to_csv(main_path+ '/' + 'moonboard' + '/' + 'generated_skeleton.csv')

for v in list_visibilities:
    df_gen[v] = 1

df_gen = df_gen.reindex(sorted(df.columns), axis=1)
df_gen

162
122
196
284
216
254
121
146
260
269


,LEFT_ANKLE.v,LEFT_ANKLE.x,LEFT_ANKLE.y,LEFT_EAR.v,LEFT_EAR.x,LEFT_EAR.y,LEFT_ELBOW.v,LEFT_ELBOW.x,LEFT_ELBOW.y,LEFT_EYE.v,...,RIGHT_PINKY.y,RIGHT_SHOULDER.v,RIGHT_SHOULDER.x,RIGHT_SHOULDER.y,RIGHT_THUMB.v,RIGHT_THUMB.x,RIGHT_THUMB.y,RIGHT_WRIST.v,RIGHT_WRIST.x,RIGHT_WRIST.y
0,1,0.678333,0.881633,1,0.721382,0.717190,1,0.689672,0.745762,1,...,0.738776,1,0.770087,0.744066,1,0.840,0.738776,1,0.840,0.738776
1,1,0.678333,0.881633,1,0.720821,0.716727,1,0.688840,0.744900,1,...,0.738776,1,0.769704,0.743733,1,0.840,0.738776,1,0.840,0.738776
2,1,0.678333,0.881633,1,0.720260,0.716263,1,0.688008,0.744037,1,...,0.738776,1,0.769321,0.743400,1,0.840,0.738776,1,0.840,0.738776
3,1,0.678333,0.881633,1,0.719698,0.715799,1,0.687176,0.743174,1,...,0.738776,1,0.768938,0.743067,1,0.840,0.738776,1,0.840,0.738776
4,1,0.678333,0.881633,1,0.719137,0.715336,1,0.686344,0.742312,1,...,0.738776,1,0.768555,0.742734,1,0.840,0.738776,1,0.840,0.738776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,1,0.533333,0.587755,1,0.800253,0.090713,1,0.814395,0.098304,1,...,0.090816,1,0.828714,0.135609,1,0.855,0.090816,1,0.855,0.090816
2027,1,0.533333,0.587755,1,0.801064,0.090585,1,0.815473,0.097806,1,...,0.090816,1,0.829236,0.135547,1,0.855,0.090816,1,0.855,0.090816
2028,1,0.533333,0.587755,1,0.801876,0.090458,1,0.816551,0.097308,1,...,0.090816,1,0.829759,0.135486,1,0.855,0.090816,1,0.855,0.090816
2029,1,0.533333,0.587755,1,0.802688,0.090330,1,0.817630,0.096811,1,...,0.090816,1,0.830281,0.135425,1,0.855,0.090816,1,0.855,0.090816


In [ ]:
draw_predicted_skeleton(video, df_pred=df_gen, frame_step = 5, drawing_with_MP=True, extension = '_GEN_spatial.mp4')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing frame 0
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
Processing frame 500
Processing frame 600
Processing frame 700
Processing frame 800
Processing frame 900
Processing frame 1000
Processing frame 1100
Processing frame 1200
Processing frame 1300
Processing frame 1400
Processing frame 1500
Processing frame 1600
Processing frame 1700
Processing frame 1800
Processing frame 1900
Processing frame 2000
